In [ ]:
# # Add at the start of your notebook
# import sys
# import os

# # Get absolute path to parent directory
# parent_dir = os.path.dirname(os.getcwd())
# sys.path.append(parent_dir)

In [14]:
# from scripts.hf_scraper_script.py import HFPaperScraper

In [ ]:
def get_paper_urls(date=None):
    
    base_url = "https://huggingface.co/papers"
    
    if date:
        url = f"{base_url}?date={date}"
    else:
        url = base_url
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    paper_links = soup.find_all('a', href=lambda x: x and '/papers/' in x)
    
    paper_urls = set()
    for link in paper_links:
        full_url = f"https://huggingface.co{link['href']}"
        paper_urls.add(full_url)
    
    return list(paper_urls)

paper_urls = get_paper_urls()

arxiv_paper_links = []

for url in paper_urls:
    # print(url)
    paper_response = requests.get(url)
    paper_soup = BeautifulSoup(paper_response.text, 'html.parser')
    paper_arxiv_link = paper_soup.find('a', href=lambda x: x and 'arxiv.org' in x)
    arxiv_paper_links.append(paper_arxiv_link['href'])

arxiv_paper_links = [link.replace('/abs/', '/pdf/') for link in arxiv_paper_links]

In [3]:
len(set(arxiv_paper_links))

16

In [4]:
def get_pdf_content(pdf_url):
    response = requests.get(pdf_url)
    pdf = PyPDF2.PdfReader(BytesIO(response.content))
    content = []
    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        content.append(page.extract_text())
    return content

In [41]:
paper_content = get_pdf_content(arxiv_paper_links[0])

In [44]:
paper_content[0]

'Preprint. Under review.\nLEARNING VIDEO REPRESENTATIONS\nWITHOUT NATURAL VIDEOS\nXueyang Yu1Xinlei Chen2Yossi Gandelsman3\n1ShanghaiTech University\n2Meta AI\n3University of California, Berkeley\nABSTRACT\nIn this paper, we show that useful video representations can be learned from\nsynthetic videos and natural images, without incorporating natural videos in the\ntraining. We propose a progression of video datasets synthesized by simple genera-\ntive processes, that model a growing set of natural video properties (e.g. motion,\nacceleration, and shape transformations). The downstream performance of video\nmodels pre-trained on these generated datasets gradually increases with the dataset\nprogression. A VideoMAE model pre-trained on our synthetic videos closes 97.2%\nof the performance gap on UCF101 action classification between training from\nscratch and self-supervised pre-training from natural videos, and outperforms\nthe pre-trained model on HMDB51. Introducing crops of static ima